In [0]:
!pip install neo4j graphdatascience

In [0]:
%restart_python

In [0]:
from pyspark.sql import SparkSession

url = "neo4j+s://0489c84a.databases.neo4j.io"
username = 'neo4j'
password = '1DB0qlHazAtJgm89k7IzQ5NVgCXvUkc0vKvCdQxZwoo'
dbname = 'neo4j'

spark = (
    SparkSession.builder.config("neo4j.url", url)
    .config("neo4j.authentication.basic.username", username)
    .config("neo4j.authentication.basic.password", password)
    .config("neo4j.database", dbname)
    .getOrCreate()
)

In [0]:
from graphdatascience.session import DbmsConnectionInfo, GdsSessions, AuraAPICredentials, SessionMemory
neo4j_aura_project_id = "ac7d1091-e574-5b61-842c-1fa087ca4a48"
sessions = GdsSessions(api_credentials=AuraAPICredentials("tRdrthUOZsp9k58mnjFjMthSXhUG7ouW", "wJt_Vxa22ATaH2aAO_bFKz6sZYBiIu7eOLcDOG1H9HNzrivKtizkp-5QAnQiRiva",project_id=neo4j_aura_project_id))
gds = sessions.get_or_create(
    session_name="gs-gds-session",
    memory=SessionMemory.m_4GB,
    db_connection=DbmsConnectionInfo(url,username, password)
)

gds.list()

In [0]:
gds.run_cypher(
    """
    CREATE
     (u1:User {name: 'Mats'}),
     (u2:User {name: 'Florentin'}),
     (p1:Product {name: 'ice cream', cost: 4.2}),
     (p2:Product {name: 'computer', cost: 13.37})

    CREATE
     (u1)-[:KNOWS {since: 2020}]->(u2),
     (u2)-[:BOUGHT {price: 7474}]->(p1),
     (u1)-[:BOUGHT {price: 1337}]->(p2)
    """
)

G, result = gds.graph.project(
    graph_name="my-graph",
    query="""
    CALL {
        MATCH (u1:User)
        OPTIONAL MATCH (u1)-[r:KNOWS]->(u2:User)
        RETURN u1 AS source, r AS rel, u2 AS target, {} AS sourceNodeProperties, {} AS targetNodeProperties
        UNION
        MATCH (p:Product)
        OPTIONAL MATCH (p)<-[r:BOUGHT]-(user:User)
        RETURN user AS source, r AS rel, p AS target, {} AS sourceNodeProperties, {cost: p.cost} AS targetNodeProperties
    }
    RETURN gds.graph.project.remote(source, target, {
      sourceNodeProperties: sourceNodeProperties,
      targetNodeProperties: targetNodeProperties,
      sourceNodeLabels: labels(source),
      targetNodeLabels: labels(target),
      relationshipType: type(rel),
      relationshipProperties: properties(rel)
    })
    """,
)

In [0]:
gds.pageRank.mutate(G, mutateProperty="pr")
gds.fastRP.mutate(G, featureProperties=["pr"], embeddingDimension=2, nodeSelfInfluence=0.1, mutateProperty="embedding")

# Stream the results back together with the `name` property fetched from the database
gds.graph.nodeProperties.stream(G, db_node_properties=["name"], node_properties=["pr", "embedding"])

In [0]:
gds.graph.nodeProperties.write(G, "embedding")

In [0]:
spark.sql("USE CATALOG data_pioneers")

# Now you can r ead the table without the catalog prefix
tableDF = spark.read.table("c360.churn_users")

tableDF